In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import ensemble
from xgboost import XGBClassifier
import time
import pandas as pd
import cv2
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:

train = pd.read_csv('fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist_test.csv')
train.isnull().any().sum()
test.isnull().any().sum()

0

In [4]:
train.shape

(60000, 785)

In [5]:
test.shape

(10000, 785)

In [6]:
df_train = train.copy()
df_test = test.copy()

In [7]:
X_train= df_train.drop(['label'],axis = 1)
X_test = df_train['label']
y_test = df_test.drop(['label'],axis = 1)

X_train = X_train.astype('float32')
y_test = y_test.astype('float32')
X_train /= 255.0
y_test /=255.0

In [8]:
seed = 99
np.random.seed(seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, X_test, test_size=0.1, random_state = seed)

In [ ]:
y_val

48993    1
38217    4
55751    2
12551    7
32669    0
        ..
27039    5
41069    1
28878    6
30385    0
10365    9
Name: label, Length: 6000, dtype: int64

In [9]:
y_val = y_val.values # change to array for mAP

In [ ]:
# # Import necessary libraries
# from sklearn.svm import SVC
# from sklearn.decomposition import PCA

# # Define hyperparameters
# n_components = 100  # Just an example, you may need to adjust this
# gamma = 0.1  # Just an example, you may need to adjust this
# C = 1.0  # Just an example, you may need to adjust this

# # Perform PCA on the training data
# pca = PCA(n_components=n_components)
# pca.fit(X_train)

# # Transform the training data
# X_train_pca = pca.transform(X_train)

# # Initialize the SVM classifier
# svm = SVC(gamma=gamma, C=C, kernel="rbf", probability=True)

# # Fit the classifier on a subset of the training data
# svm.fit(X_train_pca[:500], y_train[:500])  # Here we use only the first 500 samples

# # Predict the labels of the training data
# preds = svm.predict(X_train_pca[:500])

# # Print the shape and content of the preds
# print("Shape of preds:", preds.shape)
# print("Content of preds:", preds)


Shape of preds: (500,)
Content of preds: [0 2 1 5 8 3 9 9 6 0 5 9 2 2 7 6 8 9 0 4 0 6 3 6 9 8 7 7 3 3 2 3 6 4 3 9 5
 1 5 4 4 5 9 4 5 1 7 6 8 5 6 3 2 7 7 1 2 9 6 8 5 6 4 5 0 5 2 7 0 8 6 5 7 3
 9 9 3 8 2 6 6 9 8 0 8 2 8 1 6 7 3 9 9 1 5 8 5 4 2 4 4 6 5 4 1 9 2 5 7 7 2
 0 7 5 5 3 4 0 2 2 5 6 8 3 5 7 8 9 4 0 3 3 4 6 9 9 1 9 5 7 5 7 7 8 4 6 9 2
 7 0 1 4 1 4 1 1 2 8 5 4 7 4 0 0 3 2 9 7 5 1 6 7 8 4 8 4 9 2 7 9 3 8 8 0 7
 5 0 5 2 9 9 4 6 3 3 9 9 9 9 6 6 0 7 3 6 8 1 4 8 4 1 0 5 2 9 1 4 5 6 2 2 6
 4 9 6 6 3 3 3 7 5 5 4 3 3 9 6 1 2 9 8 8 1 7 2 8 5 5 1 9 1 2 3 6 7 0 2 5 8
 4 7 7 4 4 5 9 9 0 9 6 4 4 9 0 3 9 7 9 0 4 3 6 7 3 5 4 1 5 3 4 1 1 4 8 4 8
 0 9 2 9 7 7 4 6 0 6 5 0 4 2 8 2 5 1 1 0 7 5 3 9 9 6 9 9 3 8 8 0 8 3 4 2 9
 1 3 1 8 8 1 1 5 4 4 1 2 3 7 2 0 4 4 1 0 6 3 0 0 3 8 9 4 6 9 8 9 8 1 7 1 9
 6 8 2 5 1 7 8 3 4 2 4 8 7 5 1 6 1 0 5 3 4 1 5 2 7 2 3 0 2 3 6 2 5 5 0 8 6
 8 4 5 1 5 5 5 0 0 3 4 6 1 9 5 7 2 1 8 8 6 8 1 6 7 0 3 6 9 4 3 5 8 7 9 2 4
 5 1 7 4 3 9 3 2 1 1 8 3 1 0 3 0 2 0 2 7 9 2 9 7 4 9 7 1 8 

In [ ]:
# X_train.fillna(X_train.mean(), inplace=True)

In [10]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from collections import Counter

def objective(trial):
    # suggest methods are used to set the range of hyperparameters
    n_components = trial.suggest_int('n_components', 181, 600)
    gamma = trial.suggest_loguniform('gamma', 1e-3, 1e+2)
    C = trial.suggest_loguniform('C', 1e+0, 1e+4)

    pca = PCA(n_components=n_components)
    svm = SVC(gamma=gamma, C=C, kernel="poly", probability=True)

    # PCA fitting and transformation should be done inside the objective function
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_val_pca = pca.transform(X_val)

    # Use transformed data to train the classifier
    svm.fit(X_train_pca, y_train)

    # Predict the validation data
    preds = svm.predict(X_val_pca)

    ## mAP calculation
    AP = []
    num_class = 10

    # Count the number of each class in preds
    predict_label_count_dict = Counter(preds)
    predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

    # For each class
    for c, freq in predict_label_count_dict.items() :
        TP = 0
        FN = 0

        temp_precision = []
        temp_recall = []

        for i in range(len(preds)):
            # Calculate TP and FN
            if y_val[i] == c and preds[i] == c :
                TP += 1
            elif y_val[i] != c and preds[i] == c :
                FN += 1

            # Calculate precision and recall
            if TP+FN != 0:
                temp_precision.append(TP/(TP+FN))
                temp_recall.append(TP/freq)

        # Save the AP value of each class to AP array
        AP.append(auc(temp_recall, temp_precision))

    # Calculate mAP
    mAP = sum(AP) / num_class

    return mAP

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)


[I 2023-11-17 10:16:32,666] A new study created in memory with name: no-name-48a2b85f-7232-4159-9ef0-01bc7960559c
<ipython-input-11-50f0540b0aa1>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-3, 1e+2)
<ipython-input-11-50f0540b0aa1>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e+0, 1e+4)
[I 2023-11-17 10:37:47,424] Trial 0 finished with value: 0.8171778869597365 and parameters: {'n_components': 509, 'gamma': 5.001270433122445, 'C': 537.8832253017721}. Best is trial 0 with value: 0.8171778869597365.
<ipython-input-11-50f0540b0aa1>:11: FutureWarning: suggest_loguniform has been deprec

In [ ]:


# 최적의 하이퍼파라미터 출력
print('Best trial:')
trial = study.best_trial
print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study,
    params=[
        "n_components",
       "svc_c",
       "svc_gamma"
    ],
)

In [ ]:
# svc = SVC(gamma='scale',kernel='rbf',C=8)
# svc.fit(X_train_PCA1,y_train)

SVC(C=8)

In [ ]:
# y_pred_svc = svc.predict(X_test_PCA1)
# y_train_svc = svc.predict(X_train_PCA1)

In [ ]:
# svc_train = metrics.accuracy_score(y_train,y_train_svc)
# svc_accuracy = metrics.accuracy_score(y_val, y_pred_svc)

# print("Train Accuracy score: {}".format(svc_train))
# print("Test Accuracy score: {}".format(svc_accuracy))
# print(metrics.classification_report(y_val, y_pred_svc))

Train Accuracy score: 0.9763333333333334
Test Accuracy score: 0.9101666666666667
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       574
           1       0.99      0.99      0.99       596
           2       0.83      0.84      0.84       573
           3       0.91      0.94      0.93       624
           4       0.86      0.86      0.86       635
           5       0.99      0.96      0.98       632
           6       0.77      0.73      0.75       600
           7       0.94      0.97      0.96       604
           8       0.97      0.98      0.98       598
           9       0.97      0.96      0.97       564

    accuracy                           0.91      6000
   macro avg       0.91      0.91      0.91      6000
weighted avg       0.91      0.91      0.91      6000

